In [ ]:
#| default_exp handlers.tepco

# TEPCO 
> Data pipeline (handler) to convert TEPCO dataset ([Source](https://radioactivity.nsr.go.jp/ja/list/349/list-1.html)) to `NetCDF` format

## Packages import

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#| export
import pandas as pd
import re
import numpy as np
from typing import Tuple
from datetime import datetime
import fastcore.all as fc
from tqdm import tqdm
from collections import defaultdict

from marisco.callbacks import (
    Callback, 
    Transformer,
    EncodeTimeCB, 
    SanitizeLonLatCB,
    EncodeTimeCB, 
    )

from marisco.utils import has_valid_varname
from marisco.configs import nc_tpl_path, cfg
from marisco.encoders import NetCDFEncoder

from marisco.metadata import (
    GlobAttrsFeeder, 
    BboxCB,
    DepthRangeCB, 
    TimeRangeCB,
    ZoteroCB, 
    KeyValuePairCB    
    )

## Configuration & file paths

In [ ]:
#| exports
fname_coastal_water = 'https://radioactivity.nra.go.jp/cont/en/results/sea/coastal_water.csv'
fname_clos1F = 'https://radioactivity.nra.go.jp/cont/en/results/sea/close1F_water.xlsx'
fname_iaea_orbs = 'https://raw.githubusercontent.com/RML-IAEA/iaea.orbs/refs/heads/main/stations/station_points.csv'

fname_out = '../../_data/output/tepco.nc'

## Load data

We here load the data from the [NRA (Nuclear Regulatory Authority)](https://radioactivity.nra.go.jp/en/results) website. For the moment, we only process radioactivity concentration data in the seawater around Fukushima Dai-ichi NPP [TEPCO] (`coastal_water.csv`) and in the `close1F_water.xlsx` file.

In near future, MARIS will provide a dedicated handler for all related [ALPS data](https://radioactivity.nra.go.jp/en/results#sec-12) including measurements not only provided by TEPCO but also MOE, NRA, MLITT and Fukushima Prefecture.



:::{.callout-tip}

**FEEDBACK TO DATA PROVIDER**: The **coastal_water.csv** file contains two sections: the measurements and the locations. We identify below the line number where the locations begin. A single point of truth for the location of the stations would ease the processing in future.

:::

In [ ]:
#| exports
def find_location_section(df, 
                          col_idx=0,
                          pattern='Sampling point number'
                          ):
    "Find the line number where location data begins."
    mask = df.iloc[:, col_idx] == pattern
    indices = df[mask].index
    return indices[0] if len(indices) > 0 else -1

In [ ]:
#| eval: false
find_location_section(pd.read_csv(fname_coastal_water, low_memory=False))

27483

:::{.callout-tip}

**FEEDBACK TO DATA PROVIDER**: Distinct parsing of the time from `coastal_water.csv` and `close1F_water.xlsx` files are required. Indeed:

- `coastal_water.csv` uses the format `YYYY/MM/DD` in the `Sampling  HH:MM` and 
- `close1F_water.xlsx` uses the format `YYYY-MM-DD HH:MM:SS`.

:::

In [ ]:
#| exports
def fix_sampling_time(x):
    if pd.isna(x): 
        return '00:00:00'
    else:
        hour, min =  x.split(':')[:2]
        return f"{hour if len(hour) == 2 else '0' + hour}:{min}:00"

In [ ]:
#| exports
def get_coastal_water_df(fname_coastal_water):
    "Get the measurements dataframe from the `coastal_water.csv` file."
    
    locs_idx = find_location_section(pd.read_csv(fname_coastal_water, 
                                      skiprows=0, low_memory=False))
    
    df = pd.read_csv(fname_coastal_water, skiprows=1, 
                     nrows=locs_idx - 1,
                     low_memory=False)
    df.dropna(subset=['Sampling point number'], inplace=True)
    df['Sampling time'] = df['Sampling time'].map(fix_sampling_time)
    
    df['TIME'] = df['Sampling date'].replace('-', '/') + ' ' + df['Sampling time']
    
    df = df.drop(columns=['Sampling date', 'Sampling time'])
    return df

In [ ]:
#| eval: false
df_coastal_water = get_coastal_water_df(fname_coastal_water)
df_coastal_water.tail()

,Sampling point number,Collection layer of seawater,131I radioactivity concentration (Bq/L),131I detection limit (Bq/L),134Cs radioactivity concentration (Bq/L),134Cs detection limit (Bq/L),137Cs radioactivity concentration (Bq/L),137Cs detection limit (Bq/L),132I radioactivity concentration (Bq/L),132I detection limit (Bq/L),...,54Mn radioactivity concentration (Bq/L),54Mn detection limit (Bq/L),3H radioactivity concentration (Bq/L),3H detection limit (Bq/L),125Sb radioactivity concentration (Bq/L),125Sb detection limit (Bq/L),105Ru radioactivity concentration (Bq/L),105Ru detection limit (Bq/L),Unnamed: 49,TIME
27475,T-D5,上層,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,ND,8.0E+00,NaN,NaN,NaN,NaN,NaN,2024/10/14 07:42:00
27476,T-D5,上層,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,ND,8.3E+00,NaN,NaN,NaN,NaN,NaN,2024/10/21 07:44:00
27477,T-S8,上層,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,ND,7.5E+00,NaN,NaN,NaN,NaN,NaN,2024/10/21 07:38:00
27478,T-S3,上層,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,ND,6.8E+00,NaN,NaN,NaN,NaN,NaN,2024/10/25 09:42:00
27479,T-S4,上層,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,ND,6.8E+00,NaN,NaN,NaN,NaN,NaN,2024/10/25 10:18:00


:::{.callout-tip}

**FEEDBACK TO DATA PROVIDER** 

Identification of the stations location requires three distinct files:

    - the second section of the coastal_water.csv file
    - the R6zahyo.pdf file further processed by https://github.com/RML-IAEA/iaea.orbs
    - the second sections of all sheets of close1F_water.xlsx file
  
All files and sheets required to look up the location of the stations.

:::

In [ ]:
#| exports
def get_locs_coastal_water(fname_coastal_water):
    locs_idx = find_location_section(pd.read_csv(fname_coastal_water, 
                                      skiprows=0, low_memory=False))
    
    df = pd.read_csv(fname_coastal_water, skiprows=locs_idx+1, 
                     low_memory=False).iloc[:, :3]
    
    df.columns = ['station', 'LON', 'LAT']
    df.dropna(subset=['LAT'], inplace=True)
    df['org'] = 'coastal_seawater.csv'
    return df

In [ ]:
#| eval: false
df_locs_coastal_water = get_locs_coastal_water(fname_coastal_water)
df_locs_coastal_water.head()

,station,LON,LAT,org
0,T-0,37.42,141.04,coastal_seawater.csv
1,T-11,37.24,141.05,coastal_seawater.csv
2,T-12,37.15,141.04,coastal_seawater.csv
3,T-13-1,37.64,141.04,coastal_seawater.csv
4,T-14,37.55,141.06,coastal_seawater.csv


In [ ]:
#| eval: false
df_locs_coastal_water['station'].unique()

array(['T-0', 'T-11', 'T-12', 'T-13-1', 'T-14', 'T-17-1', 'T-18', 'T-20',
       'T-22', 'T-3', 'T-4', 'T-4-1', 'T-4-2', 'T-5', 'T-6', 'T-7', 'T-A',
       'T-B', 'T-B1', 'T-B2', 'T-B3', 'T-B4', 'T-C', 'T-D', 'T-D1',
       'T-D5', 'T-D9', 'T-E', 'T-E1', 'T-Z', 'T-MG6', 'T-S1', 'T-S7',
       'T-H1', 'T-S2', 'T-S6', 'T-M10', 'T-MA', 'T-S3', 'T-S4', 'T-S8',
       'T-MG4', 'T-G4', 'T-MG5', 'T-MG1', 'T-MG0', 'T-MG3', 'T-MG2'],
      dtype=object)

:::{.callout-tip}

**FEEDBACK TO DATA PROVIDER**: Data contained in the **close1F_water.xlsx** file are spread in several sheets (one per station). Each sheet further contains two sections: the measurements and the locations. 

For each sheet, we have to identify the line number where to split both measurements and the location. We then need to further iterate over all sheets to concatenate the results.

:::

In [ ]:
#| exports
def get_clos1F_df(fname_clos1F):
    "Get measurements dataframe from close1F_water.xlsx file and parse datetime."
    excel_file = pd.ExcelFile(fname_clos1F)
    dfs = {}
    
    for sheet_name in tqdm(excel_file.sheet_names):
        locs_idx = find_location_section(pd.read_excel(excel_file, 
                                                       sheet_name=sheet_name,
                                                       skiprows=1))
        df = pd.read_excel(excel_file, 
                   sheet_name=sheet_name, 
                   skiprows=1,
                   nrows=locs_idx-1)
        
        df.dropna(subset=['Sampling point number'], inplace=True)
        df['Sampling date'] = df['Sampling date']\
            .astype(str)\
            .apply(lambda x: x.split(' ')[0]\
            .replace('-', '/'))
            
        dfs[sheet_name] = df
    
    df = pd.concat(dfs.values(), ignore_index=True)
    df.dropna(subset=['Sampling date'], inplace=True)
    df['TIME'] = df['Sampling date'] + ' ' + df['Sampling time'].astype(str)
    df = df.drop(columns=['Sampling date', 'Sampling time'])
    return df

In [ ]:
#| eval: false
df = get_clos1F_df(fname_clos1F); df.head()

100%|██████████| 11/11 [00:06<00:00,  1.69it/s]


,Sampling point number,134Cs radioactivity concentration (Bq/L),134Cs detection limit (Bq/L),137Cs radioactivity concentration (Bq/L),137Cs detection limit (Bq/L),Total beta radioactivity concentration (Bq/L),Total beta detection limit (Bq/L),3H radioactivity concentration (Bq/L),3H detection limit (Bq/L),Collection layer of seawater,...,106Ru detection limit (Bq/L),60Co radioactivity concentration (Bq/L),60Co detection limit (Bq/L),95Zr radioactivity concentration (Bq/L),95Zr detection limit (Bq/L),99Mo radioactivity concentration (Bq/L),99Mo detection limit (Bq/L),105Ru radioactivity concentration (Bq/L),105Ru detection limit (Bq/L),TIME
0,T-0-1,ND,1.5,ND,1.4,ND,18.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013/08/14 08:17:00
1,T-0-1,NaN,NaN,NaN,NaN,NaN,NaN,4.7,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013/08/14 08:17:00
2,T-0-1,ND,1.1,ND,1.4,ND,20.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013/08/21 08:09:00
3,T-0-1,NaN,NaN,NaN,NaN,NaN,NaN,ND,2.9,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013/08/21 08:09:00
4,T-0-1,ND,0.66,ND,0.49,ND,17.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013/08/27 08:14:00


In [ ]:
#| eval: false
df['Sampling point number'].unique()

array(['T-0-1', 'T-0-1A', 'T-0-2', 'T-0-3', 'T-0-3A', 'T-1', 'T-2',
       'T-2-1', 'T-A1', 'T-A2', 'T-A3'], dtype=object)

In [ ]:
#| exports
def get_locs_clos1F(fname_clos1F):
    "Get locations dataframe from close1F_water.xlsx file from each sheets."
    excel_file = pd.ExcelFile(fname_clos1F)
    dfs = {}
    
    for sheet_name in tqdm(excel_file.sheet_names):
        locs_idx = find_location_section(pd.read_excel(excel_file, 
                                                       sheet_name=sheet_name,
                                                       skiprows=1))
        df = pd.read_excel(excel_file, 
                           sheet_name=sheet_name, 
                           skiprows=locs_idx+2)
            
        dfs[sheet_name] = df
    
    df = pd.concat(dfs.values(), ignore_index=True).iloc[:, :3]
    df.dropna(subset=['Sampling coordinate North latitude (Decimal)'], inplace=True)    
    df.columns = ['station', 'LON', 'LAT']
    df['org'] = 'close1F.csv'
    return df

In [ ]:
df_locs_clos1F = get_locs_clos1F(fname_clos1F)
df_locs_clos1F.head()

100%|██████████| 11/11 [00:05<00:00,  1.85it/s]


,station,LON,LAT,org
0,T-0-1,37.43,141.04,close1F.csv
11,T-0-1A,37.43,141.05,close1F.csv
22,T-0-2,37.42,141.05,close1F.csv
33,T-0-3,37.42,141.04,close1F.csv
44,T-0-3A,37.42,141.05,close1F.csv


:::{.callout-tip}

**FEEDBACK TO DATA PROVIDER**: In theory all locations are supposed to be provided in the [R6zahyo.pdf](https://radioactivity.nra.go.jp/cont/en/results/sea/R6zahyo.pdf) file. This file is further processed by https://github.com/RML-IAEA/iaea.orbs and the result is provided in the `station_points.csv` file. 

However, this file does not contain all locations refered to in both **coastal_water.csv** and **close1F_water.xlsx** files.

:::

In [ ]:
#| exports
def get_locs_orbs(fname_iaea_orbs):
    df = pd.read_csv(fname_iaea_orbs)
    df.columns = ['org', 'station', 'LON', 'LAT']
    return df

In [ ]:
#| eval: false
df_locs_orbs = get_locs_orbs(fname_iaea_orbs)
df_locs_orbs.head()

,org,station,LON,LAT
0,MOE,E-31,141.727667,39.059167
1,MOE,E-32,141.635667,38.996000
2,MOE,E-37,141.948611,39.259167
3,MOE,E-38,141.755000,39.008333
4,MOE,E-39,141.766667,38.991667


In [ ]:
#| exports
def concat_locs(dfs):
    "Concatenate and drop duplicates from coastal_seawater.csv and iaea_orbs.csv (kept)"
    df = pd.concat(dfs)
    # Group by org to be used for sorting
    df['org_grp'] = df['org'].apply(
        lambda x: 1 if x == 'coastal_seawater.csv' else 2 if x == 'close1F.csv' else 0)
    df.sort_values('org_grp', ascending=True, inplace=True)
    # Drop duplicates and keep orbs data first
    df.drop_duplicates(subset='station', keep='first', inplace=True)
    df.drop(columns=['org_grp'], inplace=True)
    df.sort_values('station', ascending=True, inplace=True)
    return df

In [ ]:
#| eval: false
# df_locs = concat_locs(df_locs_coastal_water, df_locs_orbs)
df_locs = concat_locs([df_locs_clos1F, df_locs_coastal_water, df_locs_orbs])
df_locs.head()

,station,LON,LAT,org
214,C-P1,139.863333,35.425000,NRA
215,C-P2,139.863333,35.401667,NRA
216,C-P3,139.881667,35.370000,NRA
217,C-P4,139.846667,35.356667,NRA
218,C-P5,139.800000,35.343333,NRA


In [ ]:
#| exports
def align_dfs(df_from, df_to):
    "Align columns structure of df_from to df_to."
    df = defaultdict()    
    for c in df_to.columns:
        df[c] = df_from[c].values if c in df_from.columns else np.NAN
    return pd.DataFrame(df)

In [ ]:
# | eval: false
align_dfs(df_clos1F, df_coastal_water).head()

,Sampling point number,Collection layer of seawater,131I radioactivity concentration (Bq/L),131I detection limit (Bq/L),134Cs radioactivity concentration (Bq/L),134Cs detection limit (Bq/L),137Cs radioactivity concentration (Bq/L),137Cs detection limit (Bq/L),132I radioactivity concentration (Bq/L),132I detection limit (Bq/L),...,54Mn radioactivity concentration (Bq/L),54Mn detection limit (Bq/L),3H radioactivity concentration (Bq/L),3H detection limit (Bq/L),125Sb radioactivity concentration (Bq/L),125Sb detection limit (Bq/L),105Ru radioactivity concentration (Bq/L),105Ru detection limit (Bq/L),Unnamed: 49,TIME
0,T-0-1,NaN,NaN,NaN,ND,1.50,ND,1.40,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,T-0-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,T-0-1,NaN,NaN,NaN,ND,1.10,ND,1.40,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,T-0-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,ND,2.9,NaN,NaN,NaN,NaN,NaN,NaN
4,T-0-1,NaN,NaN,NaN,ND,0.66,ND,0.49,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#| exports
def concat_dfs(df_coastal_water, df_clos1F):
    "Concatenate and drop duplicates from coastal_seawater.csv and close1F_water.xlsx (kept)"
    df_clos1F = align_dfs(df_clos1F, df_coastal_water)
    df = pd.concat([df_coastal_water, df_clos1F])
    return df

In [ ]:
#| eval: false
df_meas = concat_dfs(df_coastal_water, df_clos1F)
df_meas.head()

,Sampling point number,Collection layer of seawater,131I radioactivity concentration (Bq/L),131I detection limit (Bq/L),134Cs radioactivity concentration (Bq/L),134Cs detection limit (Bq/L),137Cs radioactivity concentration (Bq/L),137Cs detection limit (Bq/L),132I radioactivity concentration (Bq/L),132I detection limit (Bq/L),...,54Mn radioactivity concentration (Bq/L),54Mn detection limit (Bq/L),3H radioactivity concentration (Bq/L),3H detection limit (Bq/L),125Sb radioactivity concentration (Bq/L),125Sb detection limit (Bq/L),105Ru radioactivity concentration (Bq/L),105Ru detection limit (Bq/L),Unnamed: 49,TIME
0,T-3,NaN,1.1E+03,1.3E+01,4.8E+01,9.2E+00,5.3E+01,8.8E+00,1.6E+02,44.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011/3/21 23:15:00
1,T-4,NaN,6.6E+02,1.2E+01,3.1E+01,8.7E+00,3.3E+01,8.3E+00,1.2E+02,41.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011/3/21 23:45:00
2,T-3,NaN,1.1E+03,2.0E+01,4.6E+01,1.4E+01,4.0E+01,1.4E+01,ND,88.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011/3/22 14:28:00
3,T-4,NaN,6.7E+02,1.9E+01,3.9E+01,1.1E+01,4.4E+01,1.1E+01,ND,79.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011/3/22 15:06:00
4,T-3,NaN,7.4E+02,2.7E+01,5.1E+01,2.0E+01,5.5E+01,2.0E+01,2.0E+02,58.0,...,NaN,NaN,NaN,NaN,NaN,NaN,34.0,25.0,NaN,2011/3/23 13:51:00


In [ ]:
#| exports
def georef_data(df_meas, df_locs):
    "Georeference measurements dataframe using locations dataframe."
    assert "Sampling point number" in df_meas.columns and "station" in df_locs.columns
    return pd.merge(df_meas, df_locs, how="inner", 
                    left_on='Sampling point number', right_on='station')

In [ ]:
#| eval: false
df_meas_georef = georef_data(df_meas, df_locs)
df_meas_georef.head()

,Sampling point number,Collection layer of seawater,131I radioactivity concentration (Bq/L),131I detection limit (Bq/L),134Cs radioactivity concentration (Bq/L),134Cs detection limit (Bq/L),137Cs radioactivity concentration (Bq/L),137Cs detection limit (Bq/L),132I radioactivity concentration (Bq/L),132I detection limit (Bq/L),...,125Sb radioactivity concentration (Bq/L),125Sb detection limit (Bq/L),105Ru radioactivity concentration (Bq/L),105Ru detection limit (Bq/L),Unnamed: 49,TIME,station,LON,LAT,org
0,T-3,NaN,1.1E+03,1.3E+01,4.8E+01,9.2E+00,5.3E+01,8.8E+00,1.6E+02,44.0,...,NaN,NaN,NaN,NaN,NaN,2011/3/21 23:15:00,T-3,141.026389,37.322222,TEPCO
1,T-4,NaN,6.6E+02,1.2E+01,3.1E+01,8.7E+00,3.3E+01,8.3E+00,1.2E+02,41.0,...,NaN,NaN,NaN,NaN,NaN,2011/3/21 23:45:00,T-4,141.013889,37.241667,TEPCO
2,T-3,NaN,1.1E+03,2.0E+01,4.6E+01,1.4E+01,4.0E+01,1.4E+01,ND,88.0,...,NaN,NaN,NaN,NaN,NaN,2011/3/22 14:28:00,T-3,141.026389,37.322222,TEPCO
3,T-4,NaN,6.7E+02,1.9E+01,3.9E+01,1.1E+01,4.4E+01,1.1E+01,ND,79.0,...,NaN,NaN,NaN,NaN,NaN,2011/3/22 15:06:00,T-4,141.013889,37.241667,TEPCO
4,T-3,NaN,7.4E+02,2.7E+01,5.1E+01,2.0E+01,5.5E+01,2.0E+01,2.0E+02,58.0,...,NaN,NaN,34.0,25.0,NaN,2011/3/23 13:51:00,T-3,141.026389,37.322222,TEPCO


In [ ]:
#| exports
def load_data(fname_coastal_water, fname_clos1F, fname_iaea_orbs):
    "Load, align and georeference TEPCO data"
    df_locs = concat_locs(
        [get_locs_coastal_water(fname_coastal_water), 
         get_locs_clos1F(fname_clos1F),
         get_locs_orbs(fname_iaea_orbs)])
    df_meas = concat_dfs(get_coastal_water_df(fname_coastal_water), get_clos1F_df(fname_clos1F))
    df_meas.dropna(subset=['Sampling point number'], inplace=True)
    return {'SEAWATER': georef_data(df_meas, df_locs)}

In [ ]:
#| eval: false
dfs = load_data(fname_coastal_water, fname_clos1F, fname_iaea_orbs)
dfs['SEAWATER'].head()

100%|██████████| 11/11 [00:06<00:00,  1.69it/s]


,Sampling point number,Collection layer of seawater,131I radioactivity concentration (Bq/L),131I detection limit (Bq/L),134Cs radioactivity concentration (Bq/L),134Cs detection limit (Bq/L),137Cs radioactivity concentration (Bq/L),137Cs detection limit (Bq/L),132I radioactivity concentration (Bq/L),132I detection limit (Bq/L),...,125Sb radioactivity concentration (Bq/L),125Sb detection limit (Bq/L),105Ru radioactivity concentration (Bq/L),105Ru detection limit (Bq/L),Unnamed: 49,TIME,station,LON,LAT,org
0,T-3,NaN,1.1E+03,1.3E+01,4.8E+01,9.2E+00,5.3E+01,8.8E+00,1.6E+02,44.0,...,NaN,NaN,NaN,NaN,NaN,2011/3/21 23:15:00,T-3,141.026389,37.322222,TEPCO
1,T-4,NaN,6.6E+02,1.2E+01,3.1E+01,8.7E+00,3.3E+01,8.3E+00,1.2E+02,41.0,...,NaN,NaN,NaN,NaN,NaN,2011/3/21 23:45:00,T-4,141.013889,37.241667,TEPCO
2,T-3,NaN,1.1E+03,2.0E+01,4.6E+01,1.4E+01,4.0E+01,1.4E+01,ND,88.0,...,NaN,NaN,NaN,NaN,NaN,2011/3/22 14:28:00,T-3,141.026389,37.322222,TEPCO
3,T-4,NaN,6.7E+02,1.9E+01,3.9E+01,1.1E+01,4.4E+01,1.1E+01,ND,79.0,...,NaN,NaN,NaN,NaN,NaN,2011/3/22 15:06:00,T-4,141.013889,37.241667,TEPCO
4,T-3,NaN,7.4E+02,2.7E+01,5.1E+01,2.0E+01,5.5E+01,2.0E+01,2.0E+02,58.0,...,NaN,NaN,34.0,25.0,NaN,2011/3/23 13:51:00,T-3,141.026389,37.322222,TEPCO


In [ ]:
#| eval: false
print(f"# of cols, rows: {dfs['SEAWATER'].shape}")
dfs['SEAWATER'].head()

# of cols, rows: (46421, 53)


,Sampling point number,Collection layer of seawater,131I radioactivity concentration (Bq/L),131I detection limit (Bq/L),134Cs radioactivity concentration (Bq/L),134Cs detection limit (Bq/L),137Cs radioactivity concentration (Bq/L),137Cs detection limit (Bq/L),132I radioactivity concentration (Bq/L),132I detection limit (Bq/L),...,125Sb radioactivity concentration (Bq/L),125Sb detection limit (Bq/L),105Ru radioactivity concentration (Bq/L),105Ru detection limit (Bq/L),Unnamed: 49,TIME,station,LON,LAT,org
0,T-3,NaN,1.1E+03,1.3E+01,4.8E+01,9.2E+00,5.3E+01,8.8E+00,1.6E+02,44.0,...,NaN,NaN,NaN,NaN,NaN,2011/3/21 23:15:00,T-3,141.026389,37.322222,TEPCO
1,T-4,NaN,6.6E+02,1.2E+01,3.1E+01,8.7E+00,3.3E+01,8.3E+00,1.2E+02,41.0,...,NaN,NaN,NaN,NaN,NaN,2011/3/21 23:45:00,T-4,141.013889,37.241667,TEPCO
2,T-3,NaN,1.1E+03,2.0E+01,4.6E+01,1.4E+01,4.0E+01,1.4E+01,ND,88.0,...,NaN,NaN,NaN,NaN,NaN,2011/3/22 14:28:00,T-3,141.026389,37.322222,TEPCO
3,T-4,NaN,6.7E+02,1.9E+01,3.9E+01,1.1E+01,4.4E+01,1.1E+01,ND,79.0,...,NaN,NaN,NaN,NaN,NaN,2011/3/22 15:06:00,T-4,141.013889,37.241667,TEPCO
4,T-3,NaN,7.4E+02,2.7E+01,5.1E+01,2.0E+01,5.5E+01,2.0E+01,2.0E+02,58.0,...,NaN,NaN,34.0,25.0,NaN,2011/3/23 13:51:00,T-3,141.026389,37.322222,TEPCO


In [ ]:
#| eval: false
dfs['SEAWATER']['Sampling point number'].unique()


array(['T-3', 'T-4', 'T-5', 'T-7', 'T-11', 'T-12', 'T-14', 'T-18', 'T-20',
       'T-22', 'T-MA', 'T-M10', 'T-A', 'T-D', 'T-E', 'T-B', 'T-C',
       'T-MG1', 'T-MG2', 'T-MG3', 'T-MG4', 'T-MG5', 'T-MG6', 'T-D1',
       'T-D5', 'T-D9', 'T-E1', 'T-G4', 'T-H1', 'T-S5', 'T-S6', 'T-17-1',
       'T-B3', 'T-13-1', 'T-S3', 'T-S4', 'T-B4', 'T-S1', 'T-S2', 'T-MG0',
       'T-Z', 'T-B1', 'T-B2', 'T-S7', 'T-S8', 'T-0', 'T-4-1', 'T-4-2',
       'T-6', 'T-0-1', 'T-0-1A', 'T-0-2', 'T-0-3', 'T-0-3A', 'T-1', 'T-2',
       'T-2-1', 'T-A1', 'T-A2', 'T-A3'], dtype=object)

## Fix missing values

:::{.callout-tip}

**FEEDBACK TO DATA PROVIDER**: We remap the `ND` value to `NaN`. Please confirm that this is the correct way to handle missing values.
:::


`ND` is assigned `NaN`. This needs to be confirmed.

In [ ]:
#| exports
class FixMissingValuesCB(Callback):
    "Assign `NaN` to values equal to `ND` (not detected) - to be confirmed "
    def __call__(self, tfm): 
        for k in tfm.dfs.keys():
            predicate = tfm.dfs[k] == 'ND'
            tfm.dfs[k][predicate] = np.nan

In [ ]:
#| eval: false
tfm = Transformer(dfs, cbs=[FixMissingValuesCB()])
tfm()['SEAWATER'].head()

,Sampling point number,Collection layer of seawater,131I radioactivity concentration (Bq/L),131I detection limit (Bq/L),134Cs radioactivity concentration (Bq/L),134Cs detection limit (Bq/L),137Cs radioactivity concentration (Bq/L),137Cs detection limit (Bq/L),132I radioactivity concentration (Bq/L),132I detection limit (Bq/L),...,125Sb radioactivity concentration (Bq/L),125Sb detection limit (Bq/L),105Ru radioactivity concentration (Bq/L),105Ru detection limit (Bq/L),Unnamed: 49,TIME,station,LON,LAT,org
0,T-3,NaN,1.1E+03,1.3E+01,4.8E+01,9.2E+00,5.3E+01,8.8E+00,1.6E+02,44.0,...,NaN,NaN,NaN,NaN,NaN,2011/3/21 23:15:00,T-3,141.026389,37.322222,TEPCO
1,T-4,NaN,6.6E+02,1.2E+01,3.1E+01,8.7E+00,3.3E+01,8.3E+00,1.2E+02,41.0,...,NaN,NaN,NaN,NaN,NaN,2011/3/21 23:45:00,T-4,141.013889,37.241667,TEPCO
2,T-3,NaN,1.1E+03,2.0E+01,4.6E+01,1.4E+01,4.0E+01,1.4E+01,NaN,88.0,...,NaN,NaN,NaN,NaN,NaN,2011/3/22 14:28:00,T-3,141.026389,37.322222,TEPCO
3,T-4,NaN,6.7E+02,1.9E+01,3.9E+01,1.1E+01,4.4E+01,1.1E+01,NaN,79.0,...,NaN,NaN,NaN,NaN,NaN,2011/3/22 15:06:00,T-4,141.013889,37.241667,TEPCO
4,T-3,NaN,7.4E+02,2.7E+01,5.1E+01,2.0E+01,5.5E+01,2.0E+01,2.0E+02,58.0,...,NaN,NaN,34.0,25.0,NaN,2011/3/23 13:51:00,T-3,141.026389,37.322222,TEPCO


## Remove 約 (about) character
    

:::{.callout-tip}

**FEEDBACK TO DATA PROVIDER**: We systematically remove the `約` character. Please confirm that this is the correct way to handle this. We could imagine that mentioning uncertainty would be less ambiguous in future.

:::

In [ ]:
#| exports
class RemoveJapanaseCharCB(Callback):
    "Remove 約 (about) char"
    def _transform_if_about(self, value, about_char='約'):
        if pd.isna(value): return value
        return (value.replace(about_char, '') if str(value).count(about_char) != 0 
                else value)
    
    def __call__(self, tfm): 
        for k in tfm.dfs.keys():
            cols_rdn = [c for c in tfm.dfs[k].columns if ('(Bq/L)' in c) and (tfm.dfs[k][c].dtype == 'object')]
            tfm.dfs[k][cols_rdn] = tfm.dfs[k][cols_rdn].map(self._transform_if_about)

In [ ]:
tfm = Transformer(dfs, cbs=[
    FixMissingValuesCB(),
    RemoveJapanaseCharCB()])

tfm()['SEAWATER'].sample(10)

,Sampling point number,Collection layer of seawater,131I radioactivity concentration (Bq/L),131I detection limit (Bq/L),134Cs radioactivity concentration (Bq/L),134Cs detection limit (Bq/L),137Cs radioactivity concentration (Bq/L),137Cs detection limit (Bq/L),132I radioactivity concentration (Bq/L),132I detection limit (Bq/L),...,125Sb radioactivity concentration (Bq/L),125Sb detection limit (Bq/L),105Ru radioactivity concentration (Bq/L),105Ru detection limit (Bq/L),Unnamed: 49,TIME,station,LON,LAT,org
32242,T-1,NaN,53000,86,21000,72,21000,66,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2011/04/02 08:50:00,T-1,141.034444,37.431111,TEPCO
42195,T-2,上層,NaN,NaN,NaN,0.73,NaN,0.63,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2022/05/04 06:45:00,T-2,141.033611,37.415833,TEPCO
23181,T-E,下層,NaN,NaN,NaN,7.9E-01,NaN,1.1E+00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2023/6/13 13:29:00,T-E,140.837222,35.796111,TEPCO
45249,T-A1,上層,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2023/02/13 07:49:00,T-A1,141.050761,37.440794,TEPCO
4652,T-D9,下層,NaN,NaN,2.3E-01,NaN,4.1E-01,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2013/2/15 08:18:00,T-D9,141.072167,37.333333,TEPCO
42364,T-2,上層,NaN,NaN,NaN,0.81,NaN,0.85,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2022/09/01 09:34:00,T-2,141.033611,37.415833,TEPCO
7178,T-5,上層,NaN,NaN,NaN,1.3E-03,6.4E-03,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2014/5/20 08:34:00,T-5,141.200000,37.416667,TEPCO
28474,T-0-2,NaN,NaN,NaN,NaN,0.69,NaN,0.76,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2015/06/01 09:19:00,T-0-2,141.046667,37.423333,TEPCO
19315,T-S5,上層,NaN,NaN,NaN,1.4E-03,1.1E-02,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2021/3/8 06:12:00,T-S5,141.039444,37.265000,TEPCO
4130,T-20,上層,NaN,NaN,9.4E-03,NaN,1.6E-02,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2012/11/10 06:15:00,T-20,141.000000,36.966667,TEPCO


## Fix values range string

:::{.callout-tip}

**FEEDBACK TO DATA PROVIDER**: Value ranges are provided as strings (e.g '4.0E+00<&<8.0E+00' or '1.0～2.7'). We replace them by their mean. Please confirm that this is the correct way to handle this. Again, mentioning uncertainty would be less ambiguous in future.

:::

In [ ]:
#| exports
class FixRangeValueStringCB(Callback):
    "Replace range values (e.g '4.0E+00<&<8.0E+00' or '1.0～2.7') by their mean"
    
    def _extract_and_calculate_mean(self, s):
        # For scientific notation ranges
        float_strings = re.findall(r"[+-]?\d+\.?\d*E?[+-]?\d*", s)
        if float_strings:
            float_numbers = np.array(float_strings, dtype=float)
            return float_numbers.mean()
        return s
    
    def _transform_if_range(self, value):
        if pd.isna(value): 
            return value
        value = str(value)
        # Check for both range patterns
        if '<&<' in value or '～' in value:
            return self._extract_and_calculate_mean(value)
        return value

    def __call__(self, tfm): 
        for k in tfm.dfs.keys():
            cols_rdn = [c for c in tfm.dfs[k].columns 
                       if ('(Bq/L)' in c) and (tfm.dfs[k][c].dtype == 'object')]
            tfm.dfs[k][cols_rdn] = tfm.dfs[k][cols_rdn].map(self._transform_if_range).astype(float)

In [ ]:
tfm = Transformer(dfs, cbs=[
    FixMissingValuesCB(),
    RemoveJapanaseCharCB(),
    FixRangeValueStringCB()
    ])

df_test = tfm()['SEAWATER']
df_test.sample(10)

,Sampling point number,Collection layer of seawater,131I radioactivity concentration (Bq/L),131I detection limit (Bq/L),134Cs radioactivity concentration (Bq/L),134Cs detection limit (Bq/L),137Cs radioactivity concentration (Bq/L),137Cs detection limit (Bq/L),132I radioactivity concentration (Bq/L),132I detection limit (Bq/L),...,125Sb radioactivity concentration (Bq/L),125Sb detection limit (Bq/L),105Ru radioactivity concentration (Bq/L),105Ru detection limit (Bq/L),Unnamed: 49,TIME,station,LON,LAT,org
8598,T-MG5,中層,NaN,NaN,NaN,0.0015,0.0046,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2015/1/21 08:13:00,T-MG5,141.250000,38.166667,TEPCO
28871,T-0-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2019/03/18 06:32:00,T-0-2,141.046667,37.423333,TEPCO
4193,T-D5,下層,NaN,NaN,0.044,NaN,0.0800,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2012/11/19 08:20:00,T-D5,141.072222,37.416667,TEPCO
31739,T-0-3A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2021/05/10 06:47:00,T-0-3A,141.046667,37.416111,TEPCO
40773,T-2,上層,NaN,NaN,NaN,0.4900,NaN,0.53,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2019/08/13 07:30:00,T-2,141.033611,37.415833,TEPCO
25045,T-S3,上層,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2024/6/12 09:53:00,T-S3,141.078889,37.458333,TEPCO
35608,T-1,上層,NaN,0.65,NaN,0.6700,NaN,0.70,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2018/08/20 07:00:00,T-1,141.034444,37.431111,TEPCO
18194,T-6,上層,NaN,NaN,NaN,0.0013,0.0097,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2020/6/30 09:45:00,T-6,141.040556,37.478889,TEPCO
40799,T-2,上層,NaN,NaN,NaN,0.7300,NaN,0.64,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2019/09/02 06:50:00,T-2,141.033611,37.415833,TEPCO
3658,T-D1,下層,NaN,NaN,0.025,NaN,0.0370,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2012/8/6 07:40:00,T-D1,141.072222,37.500000,TEPCO


## Select columns of interest

We select the columns of interest and in particular the elements of interest, in our case radionuclides.

In [ ]:
#| exports
common_coi = ['org', 'LON', 'LAT', 'TIME', 'station']
nuclides_pattern = '(Bq/L)'

In [ ]:
#| exports
class SelectColsOfInterestCB(Callback):
    "Select columns of interest."
    def __init__(self, common_coi, nuclides_pattern): fc.store_attr()
    def __call__(self, tfm):
        nuc_of_interest = [c for c in tfm.dfs['SEAWATER'].columns if nuclides_pattern in c]
        tfm.dfs['SEAWATER'] = tfm.dfs['SEAWATER'][self.common_coi + nuc_of_interest]

In [ ]:
#| eval: false
tfm = Transformer(dfs, cbs=[
    FixMissingValuesCB(),
    RemoveJapanaseCharCB(),
    FixRangeValueStringCB(),
    SelectColsOfInterestCB(common_coi, nuclides_pattern)
    ])

df_test = tfm()['SEAWATER'] 
df_test.sample(5)

,org,LON,LAT,TIME,station,131I radioactivity concentration (Bq/L),131I detection limit (Bq/L),134Cs radioactivity concentration (Bq/L),134Cs detection limit (Bq/L),137Cs radioactivity concentration (Bq/L),...,144Ce radioactivity concentration (Bq/L),144Ce detection limit (Bq/L),54Mn radioactivity concentration (Bq/L),54Mn detection limit (Bq/L),3H radioactivity concentration (Bq/L),3H detection limit (Bq/L),125Sb radioactivity concentration (Bq/L),125Sb detection limit (Bq/L),105Ru radioactivity concentration (Bq/L),105Ru detection limit (Bq/L)
2194,TEPCO,141.062500,37.552778,2011/12/20 09:40:00,T-14,NaN,0.65,NaN,0.9500,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5778,TEPCO,141.666667,38.300000,2013/9/6 07:55:00,T-MG2,NaN,NaN,NaN,0.0019,0.0018,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15105,TEPCO,141.042222,37.584722,2018/9/6 05:50:00,T-S1,NaN,NaN,NaN,0.0012,0.0078,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17403,TEPCO,140.603889,36.299722,2020/1/15 08:38:00,T-C,NaN,NaN,NaN,0.9200,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24502,TEPCO,141.042500,37.640833,2024/3/19 08:07:00,T-13-1,NaN,NaN,NaN,0.0014,0.0044,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Reshape: wide to long

This step is necessary to extract information such as nuclide names,  detection limint, uncertainty, ...


In [ ]:
#| exports
class WideToLongCB(Callback):
    """
    Parse TEPCO measurement columns to extract nuclide name, measurement value, 
    detection limit and uncertainty
    """
    def __init__(self): fc.store_attr()
    
    
    def _melt(self, df):
        "Melt dataframe to long format."
        return df.melt(id_vars=['LON', 'LAT', 'TIME', 'station'])
        
    def _extract_nuclide(self, text):
        words = text.split(' ')
        # Handle special cases for alpha/beta
        if len(words) >= 2 and words[1].lower() in ['alpha', 'beta']:
            return f"{words[0]} {words[1]}"
        return words[0]
    
    def _nuclide_name(self, df):
        "Extract nuclide name from nuclide names."
        df['NUCLIDE'] = df['variable'].map(self._extract_nuclide)
        return df
    
    def _type_indicator(self, df):
        "Create type indicators."
        df['is_concentration'] = df['variable'].str.contains('radioactivity concentration')
        df['is_dl'] = df['variable'].str.contains('detection limit')
        df['is_unc'] = df['variable'].str.contains('statistical error')
        return df
    
    def _unit(self, df):
        "Extract unit from nuclide names."
        df['UNIT'] = df['variable'].str.extract(r'\((.*?)\)')
        return df
    
    def _type_column(self, df):
        "Create type column."
        conditions = [
            df['is_concentration'],
            df['is_dl'],
            df['is_unc']
        ]
        choices = ['VALUE', 'DL', 'UNC']
        df['type'] = np.select(conditions, choices)
        df = df.drop(['is_concentration', 'is_dl', 'is_unc'], axis=1)
        return df
    
    def __call__(self, tfm):
        tfm.dfs['SEAWATER'] = self._melt(tfm.dfs['SEAWATER'])
        tfm.dfs['SEAWATER'] = self._nuclide_name(tfm.dfs['SEAWATER'])
        tfm.dfs['SEAWATER'] = self._type_indicator(tfm.dfs['SEAWATER'])
        tfm.dfs['SEAWATER'] = self._unit(tfm.dfs['SEAWATER'])
        tfm.dfs['SEAWATER'] = self._type_column(tfm.dfs['SEAWATER'])
        tfm.dfs['SEAWATER'] = pd.pivot_table(
            tfm.dfs['SEAWATER'],
            values='value',
            index=['LON', 'LAT', 'TIME', 'station', 'NUCLIDE', 'UNIT'],
            columns='type',
            aggfunc='first'
        ).reset_index()
        # reset the index and rename it ID
        tfm.dfs['SEAWATER'].reset_index(inplace=True)
        tfm.dfs['SEAWATER'].rename(columns={'index': 'ID'}, inplace=True)
        

In [ ]:
#| eval: false
tfm = Transformer(dfs, cbs=[
    FixMissingValuesCB(),
    RemoveJapanaseCharCB(),
    FixRangeValueStringCB(),
    SelectColsOfInterestCB(common_coi, nuclides_pattern),
    WideToLongCB()
    ])

df_test = tfm()['SEAWATER'] 
df_test.head()

type,ID,LON,LAT,TIME,station,NUCLIDE,UNIT,DL,UNC,VALUE
0,0,37.21,141.01,2012/10/16 07:25:00,T-4-1,131I,Bq/L,0.13,NaN,NaN
1,1,37.21,141.01,2012/10/16 07:25:00,T-4-1,134Cs,Bq/L,0.19,NaN,NaN
2,2,37.21,141.01,2012/10/16 07:25:00,T-4-1,137Cs,Bq/L,0.27,NaN,NaN
3,3,37.21,141.01,2012/10/2 07:30:00,T-4-1,131I,Bq/L,0.11,NaN,NaN
4,4,37.21,141.01,2012/10/2 07:30:00,T-4-1,134Cs,Bq/L,0.22,NaN,NaN


## Remap `UNIT` name to MARIS id

In [ ]:
#| exports
unit_mapping = {'Bq/L': 3}

In [ ]:
#| exports
class RemapUnitNameCB(Callback):
    """
    Remap `UNIT` name to MARIS id.
    """
    def __init__(self, unit_mapping): fc.store_attr()
    def __call__(self, tfm):
        tfm.dfs['SEAWATER']['UNIT'] = tfm.dfs['SEAWATER']['UNIT'].map(self.unit_mapping)


In [ ]:
#| eval: false
tfm = Transformer(dfs, cbs=[
    FixMissingValuesCB(),
    RemoveJapanaseCharCB(),
    FixRangeValueStringCB(),
    SelectColsOfInterestCB(common_coi, nuclides_pattern),
    WideToLongCB(),
    RemapUnitNameCB(unit_mapping)
    ])

df_test = tfm()['SEAWATER'] 
df_test.sample(5)

type,ID,LON,LAT,TIME,station,NUCLIDE,UNIT,DL,UNC,VALUE
48169,48169,141.040278,37.416111,2016/04/11 08:19,T-0-3,134Cs,3,0.79,NaN,NaN
9551,9551,140.789167,36.791667,2023/12/15 08:08:00,T-Z,134Cs,3,0.92,NaN,NaN
38881,38881,141.034444,37.431111,2016/09/22 07:10,T-1,134Cs,3,0.76,NaN,NaN
39697,39697,141.034444,37.431111,2017/06/03 06:51:00,T-1,137Cs,3,0.64,NaN,NaN
48744,48744,141.040278,37.416111,2019/02/04 06:54:00,T-0-3,3H,3,0.92,NaN,NaN


## Remap `NUCLIDE` name to MARIS id

In [ ]:
#| exports
nuclide_mapping = {
    '131I': 29,
    '134Cs': 31,
    '137Cs': 33,
    '125Sb': 24,
    'Total beta': 103,
    '238Pu': 67,
    '239Pu+240Pu': 77,
    '3H': 1,
    '89Sr': 11,
    '90Sr': 12,
    'Total alpha': 104,
    '132I': 100,
    '136Cs': 102,
    '58Co': 8,
    '105Ru': 97,
    '106Ru': 17,
    '140La': 35,
    '140Ba': 34,
    '132Te': 99,
    '60Co': 9,
    '144Ce': 37,
    '54Mn': 6
}

In [ ]:
#| exports
class RemapNuclideNameCB(Callback):
    """
    Remap `NUCLIDE` name to MARIS id.
    """
    def __init__(self, nuclide_mapping): fc.store_attr()
    def __call__(self, tfm):
        tfm.dfs['SEAWATER']['NUCLIDE'] = tfm.dfs['SEAWATER']['NUCLIDE'].map(self.nuclide_mapping)

In [ ]:
#| eval: false
tfm = Transformer(dfs, cbs=[
    FixMissingValuesCB(),
    RemoveJapanaseCharCB(),
    FixRangeValueStringCB(),
    SelectColsOfInterestCB(common_coi, nuclides_pattern),
    WideToLongCB(),
    RemapUnitNameCB(unit_mapping),
    RemapNuclideNameCB(nuclide_mapping)
    ])

df_test = tfm()['SEAWATER'] 
df_test.sample(5)

type,ID,LON,LAT,TIME,station,NUCLIDE,UNIT,DL,UNC,VALUE
73341,73341,141.072222,37.500000,2019/12/16 08:48:00,T-D1,1,3,0.34,NaN,NaN
27969,27969,141.033611,37.415833,2021/06/20 06:45:00,T-2,103,3,NaN,NaN,15.0
54430,54430,141.042222,37.584722,2015/5/21 05:38:00,T-S1,31,3,NaN,NaN,0.0035
69195,69195,141.072167,37.333333,2021/12/15 08:09:00,T-D9,33,3,NaN,NaN,0.012
80945,80945,141.200000,37.416667,2023/8/30 07:53:00,T-5,104,3,2.3,NaN,NaN


## Parse & encode time

In [ ]:
#| exports
class ParseTimeCB(Callback):
    "Parse time column from TEPCO."
    def __init__(self,
                 time_name='TIME'):
        fc.store_attr()
        
    def __call__(self, tfm):
        tfm.dfs['SEAWATER'][self.time_name] = pd.to_datetime(tfm.dfs['SEAWATER'][self.time_name], 
                                                             format='%Y/%m/%d %H:%M:%S', errors='coerce')

In [ ]:
#| eval: false
tfm = Transformer(dfs, cbs=[
    FixMissingValuesCB(),
    RemoveJapanaseCharCB(),
    FixRangeValueStringCB(),
    SelectColsOfInterestCB(common_coi, nuclides_pattern),
    WideToLongCB(),
    RemapUnitNameCB(unit_mapping),
    RemapNuclideNameCB(nuclide_mapping),
    ParseTimeCB(),
    EncodeTimeCB()
    ])

df_test = tfm()['SEAWATER'] 
df_test.sample(5)


type,ID,LON,LAT,TIME,station,NUCLIDE,UNIT,DL,UNC,VALUE
47528,47528,141.039444,37.265000,1663135860,T-S5,33,3,NaN,NaN,0.0032
50967,50967,141.040278,37.430556,1511335800,T-0-1,1,3,1.7,NaN,NaN
6768,6768,140.665556,36.506389,1337243220,T-B,31,3,1.4,NaN,NaN
23170,23170,141.033611,37.415833,1510558500,T-2,33,3,0.58,NaN,0.036
43854,43854,141.034444,37.431111,1627201080,T-1,31,3,0.7,NaN,NaN


## Sanitize coordinates

In [ ]:
#| eval: false
tfm = Transformer(dfs, cbs=[
    FixMissingValuesCB(),
    RemoveJapanaseCharCB(),
    FixRangeValueStringCB(),
    SelectColsOfInterestCB(common_coi, nuclides_pattern),
    WideToLongCB(),
    RemapUnitNameCB(unit_mapping),
    RemapNuclideNameCB(nuclide_mapping),
    ParseTimeCB(),
    EncodeTimeCB(),
    SanitizeLonLatCB()
    ])

df_test = tfm()['SEAWATER']
df_test.sample(5)

type,ID,LON,LAT,TIME,station,NUCLIDE,UNIT,DL,UNC,VALUE
52587,52587,141.040556,37.478889,1380616800,T-6,33,3,NaN,NaN,0.2
58760,58760,141.046667,37.423333,1576479720,T-0-2,1,3,0.84,NaN,NaN
9964,9964,140.837222,35.796111,1341930540,T-E,33,3,1.3,NaN,NaN
25772,25772,141.033611,37.415833,1567148400,T-2,31,3,0.67,NaN,NaN
11206,11206,141.000000,36.966667,1330671300,T-20,33,3,1.1,NaN,NaN


## Encode to NetCDF

In [ ]:
#| eval: false
tfm = Transformer(dfs, cbs=[
    FixMissingValuesCB(),
    RemoveJapanaseCharCB(),
    FixRangeValueStringCB(),
    SelectColsOfInterestCB(common_coi, nuclides_pattern),
    WideToLongCB(),
    RemapUnitNameCB(unit_mapping),
    RemapNuclideNameCB(nuclide_mapping),
    ParseTimeCB(),
    EncodeTimeCB(),
    SanitizeLonLatCB()
    ])

dfs_tfm = tfm()
tfm.logs

['Assign `NaN` to values equal to `ND` (not detected) - to be confirmed ',
 'Remove 約 (about) char',
 "Replace range values (e.g '4.0E+00<&<8.0E+00' or '1.0～2.7') by their mean",
 'Select columns of interest.',
 '\n    Parse TEPCO measurement columns to extract nuclide name, measurement value, \n    detection limit and uncertainty\n    ',
 '\n    Remap `UNIT` name to MARIS id.\n    ',
 '\n    Remap `NUCLIDE` name to MARIS id.\n    ',
 'Parse time column from TEPCO.',
 'Encode time as seconds since epoch.',
 'Drop rows with invalid longitude & latitude values. Convert `,` separator to `.` separator.']

In [ ]:
#| exports
kw = ['oceanography', 'Earth Science > Oceans > Ocean Chemistry> Radionuclides',
      'Earth Science > Human Dimensions > Environmental Impacts > Nuclear Radiation Exposure',
      'Earth Science > Oceans > Ocean Chemistry > Ocean Tracers, Earth Science > Oceans > Marine Sediments',
      'Earth Science > Oceans > Ocean Chemistry, Earth Science > Oceans > Sea Ice > Isotopes',
      'Earth Science > Oceans > Water Quality > Ocean Contaminants',
      'Earth Science > Biological Classification > Animals/Vertebrates > Fish',
      'Earth Science > Biosphere > Ecosystems > Marine Ecosystems',
      'Earth Science > Biological Classification > Animals/Invertebrates > Mollusks',
      'Earth Science > Biological Classification > Animals/Invertebrates > Arthropods > Crustaceans',
      'Earth Science > Biological Classification > Plants > Macroalgae (Seaweeds)']

In [ ]:
#| exports
def get_attrs(tfm, zotero_key, kw=kw):
    "Retrieve global attributes from MARIS dump."
    return GlobAttrsFeeder(tfm.dfs, cbs=[
        BboxCB(),
        TimeRangeCB(),
        ZoteroCB(zotero_key, cfg=cfg()),
        KeyValuePairCB('keywords', ', '.join(kw)),
        KeyValuePairCB('publisher_postprocess_logs', ', '.join(tfm.logs))
        ])()

In [ ]:
#| eval: false
get_attrs(tfm, zotero_key='JEV6HP5A', kw=kw)

{'geospatial_lat_min': '141.66666667',
 'geospatial_lat_max': '38.63333333',
 'geospatial_lon_min': '140.60388889',
 'geospatial_lon_max': '35.79611111',
 'geospatial_bounds': 'POLYGON ((140.60388889 35.79611111, 141.66666667 35.79611111, 141.66666667 38.63333333, 140.60388889 38.63333333, 140.60388889 35.79611111))',
 'time_coverage_start': '2011-03-21T14:30:00',
 'time_coverage_end': '2024-10-26T07:32:00',
 'title': "Readings of Sea Area Monitoring - Monitoring of sea water - Sea area close to TEPCO's Fukushima Daiichi NPS / Coastal area - Readings of Sea Area Monitoring [TEPCO]",
 'summary': '',
 'creator_name': '[{"creatorType": "author", "firstName": "", "lastName": "TEPCO - Tokyo Electric Power Company"}]',
 'keywords': 'oceanography, Earth Science > Oceans > Ocean Chemistry> Radionuclides, Earth Science > Human Dimensions > Environmental Impacts > Nuclear Radiation Exposure, Earth Science > Oceans > Ocean Chemistry > Ocean Tracers, Earth Science > Oceans > Marine Sediments, Eart

In [ ]:
#| exports
def encode(
    fname_out: str, # Path to the folder where the NetCDF output will be saved
    **kwargs # Additional keyword arguments
    ):
    "Encode TEPCO data to NetCDF."
    dfs = load_data(fname_coastal_water, fname_clos1F, fname_iaea_orbs)
    
    tfm = Transformer(dfs, cbs=[
        FixMissingValuesCB(),
        RemoveJapanaseCharCB(),
        FixRangeValueStringCB(),
        SelectColsOfInterestCB(common_coi, nuclides_pattern),
        WideToLongCB(),
        RemapUnitNameCB(unit_mapping),
        RemapNuclideNameCB(nuclide_mapping),
        ParseTimeCB(),
        EncodeTimeCB(),
        SanitizeLonLatCB()
    ])        
    tfm()
    encoder = NetCDFEncoder(tfm.dfs, 
                            dest_fname=fname_out, 
                            global_attrs=get_attrs(tfm, zotero_key='JEV6HP5A', kw=kw),
                            verbose=kwargs.get('verbose', False)
                            )
    encoder.encode()

In [ ]:
# encode(fname_out, verbose=True)